## Introduction to Bedrock - Generating images using Stable Diffusion
---
In this demo notebook, we demonstrate how to use the Bedrock SDK for an image generation task. We show how to use the Stable Diffusion foundational model to create images
1. Text to Image
2. Image to Image

Images in Stable Diffusion are generated by these 4 main models below
1. The CLIP text encoder;
2. The VAE decoder;
3. The UNet, and
4. The VAE_post_quant_conv

These blocks are chosen because they represent the bulk of the compute in the pipeline

see this diagram below

![SD Architecture](./images/sd.png)

#### Image prompting

Writing a good prompt can sometime be an art. It is often difficult to predict whether a certain prompt will yield a satisfactory image with a given model. However, there are certain templates that have been observed to work. Broadly, a prompt can be roughly broken down into three pieces: (i) type of image (photograph/sketch/painting etc.), (ii) description (subject/object/environment/scene etc.) and (iii) the style of the image (realistic/artistic/type of art etc.). You can change each of the three parts individually to generate variations of an image. Adjectives have been known to play a significant role in the image generation process. Also, adding more details help in the generation process.

To generate a realistic image, you can use phrases such as “a photo of”, “a photograph of”, “realistic” or “hyper realistic”. To generate images by artists you can use phrases like “by Pablo Piccaso” or “oil painting by Rembrandt” or “landscape art by Frederic Edwin Church” or “pencil drawing by Albrecht Dürer”. You can also combine different artists as well. To generate artistic image by category, you can add the art category in the prompt such as “lion on a beach, abstract”. Some other categories include “oil painting”, “pencil drawing, “pop art”, “digital art”, “anime”, “cartoon”, “futurism”, “watercolor”, “manga” etc. You can also include details such as lighting or camera lens such as 35mm wide lens or 85mm wide lens and details about the framing (portrait/landscape/close up etc.).

Note that model generates different images even if same prompt is given multiple times. So, you can generate multiple images and select the image that suits your application best.

In [2]:
%pip install -U boto3 botocore --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
awscli 1.29.42 requires botocore==1.31.42, but you have botocore 1.31.63 which is incompatible.
awscli 1.29.42 requires s3transfer<0.7.0,>=0.6.0, but you have s3transfer 0.7.0 which is incompatible.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.3 which is incompatible.
notebook 6.5.5 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.3.1 which is incompatible.
notebook 6.5.5 requires pyzmq<25,>=17, but you have pyzmq 25.1.1 which is inc

#### Now let's set up our connection to the Amazon Bedrock SDK using Boto3

In [3]:
%pip install langchain==0.0.309 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.2 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.15.0 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.0a7 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install pillow==9.5.0 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [5]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'

In [6]:
import boto3
import json
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

bedrock_client = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True  # Default. Needed for invoke_model() from the data plane
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [7]:
import io, base64
from PIL import Image

## Text to Image
In order to generate an image, a description of what needs to be generated is needed. This is called `prompt`.

You can also provide some negative prompts to guide the model to avoid certain type of outputs.

Prompt acts as the input to the model and steers the model to generate a relevant output. With Stable Diffusion XL you have the option to choose certain [style presets](https://platform.stability.ai/docs/release-notes#style-presets) as well

In [8]:
prompt = "Dog in a forest"
negative_prompts = [
    "poorly rendered", 
    "poor background details", 
    "poorly drawn dog", 
    "disfigured dog features"
    ]
style_preset = "photographic" # (photographic, digital-art, cinematic, ...)
#prompt = "photo taken from above of an italian landscape. cloud is clear with few clouds. Green hills and few villages, a lake"

`Bedrock` class implements a method `generate_image`. This method takes input a prompt and prepares a payload to be sent over to Bedrock API.
You can provide the following model inference parameters to control the repetitiveness of responses:
- prompt (string): Input text prompt for the model
- seed (int): Determines initial noise. Using same seed with same settings will create similar images.
- cfg_scale (float): Presence strength - Determines how much final image portrays prompts.
- steps (int): Generation step - How many times image is sampled. More steps may be more accurate.

As an output the Bedrock generates a `base64` encoded string respresentation of the image.

In [9]:
model = bedrock.Bedrock(bedrock_client)
base_64_img_str = model.generate_image(prompt, cfg_scale=5, seed=5450, steps=70, style_preset=style_preset)

AttributeError: module 'utils.bedrock' has no attribute 'Bedrock'

We can convert the `base64` image to a PIL image to be displayed

In [ ]:
image_1 = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))

In [ ]:
image_1

## Image to Image

Stable Diffusion let's us do some interesting stuff with our images like adding new characters or modifying scenery let's give it a try.

You can use the previously generated image or use a different one to create a base64 string to be passed on as an initial image to the model.

In [ ]:
from io import BytesIO
from base64 import b64encode

buffer = BytesIO()
image_1.save(buffer, format="JPEG")
img_bytes = buffer.getvalue()
init_image = b64encode(img_bytes).decode()

A new guiding prompt can then help the model to act on the intial image

In [ ]:
change_prompt = "add some leaves around the dog"

The `generate_image` method also accepts an additional paramter `init_image` which can be used to pass the initial image to the Stable Diffusion model on Bedrock.

In [ ]:
base_64_img_str = model.generate_image(change_prompt, init_image=init_image, seed=321, start_schedule=0.6)

In [ ]:
image_2 = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))

In [ ]:
image_2

### Summary

And play around with different prompts to see amazing results.